In [1]:

import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

import numpy as np
import imutils
import time
import cv2
import os

def generate_dataset():
    if(t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Result','Please provide complete details of the user')
    else:
        mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="12345",
        database="face"
        )
        mycursor=mydb.cursor()
        mycursor.execute("SELECT * from Authorized_user")
        myresult=mycursor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql="insert into Authorized_user(id,Name,Age,Address) values(%s,%s,%s,%s)"
        val=(id,t1.get(),t2.get(),t3.get())
        mycursor.execute(sql,val)
        mydb.commit()
        
        def detect_and_predict_mask(frame, faceNet, maskNet):

            (h, w) = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
            faceNet.setInput(blob)
            detections = faceNet.forward()
            print(detections.shape)


            faces = []
            locs = []
            preds = []
            for i in range(0, detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                if confidence > 0.5:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")
                    (startX, startY) = (max(0, startX), max(0, startY))
                    (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
                    face = frame[startY:endY, startX:endX]
                    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                    face = cv2.resize(face, (224, 224))
                    face = img_to_array(face)
                    face = preprocess_input(face)


                    faces.append(face)
                    locs.append((startX, startY, endX, endY))
                if len(faces) > 0:

                    faces = np.array(faces, dtype="float32")
                    preds = maskNet.predict(faces, batch_size=32)
            return (locs, preds)
    prototxtPath = r"face_detector\deploy.prototxt"
    weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
    maskNet = load_model("mask_detector.model")
    vs = VideoStream(src=0).start()
    print("[INFO] starting video stream...")
    while True:
        frame = vs.read()
        frame = imutils.resize(frame, width=1200)


        (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
        

        for (box, pred) in zip(locs, preds):
            (startX, startY, endX, endY) = box
            (mask, withoutMask) = pred
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
            cv2.putText(frame, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        window.geometry("800x200")
        window.mainloop()
        cv2.destroyAllWindows()
        vs.stop()







#prototxtPath = r"face_detector\deploy.prototxt"
#weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
#faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)


#maskNet = load_model("mask_detector.model")
window=tk.Tk()
window.title("Face recognition system")
window.config(background="lime")
l1=tk.Label(window,text="Name",font=("Algerian",20))
l1.grid(column=0, row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1, row=0)
l2=tk.Label(window,text="Age",font=("Algerian",20))
l2.grid(column=0, row=1)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1, row=1)

l3=tk.Label(window,text="Address",font=("Algerian",20))
l3.grid(column=0, row=2)
t3=tk.Entry(window,width=50,bd=5)
t3.grid(column=1, row=2)

b1=tk.Button(window,text="Generate dataset",font=("Algerian",20),bg="pink",fg="black",command=generate_dataset)
b1.grid(column=1,row=3)




#print("[INFO] starting video stream...")









'''
while True:

	frame = vs.read()
	frame = imutils.resize(frame, width=1200)


	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	for (box, pred) in zip(locs, preds):

		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred


		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)


		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)


	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF


	if key == ord("q"):
		break

'''

window.geometry("800x200")
window.mainloop()

[INFO] starting video stream...
(1, 1, 95, 7)
(1, 1, 101, 7)


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\sunshine\anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-1-dfeb3e8dd2f3>", line 101, in generate_dataset
    window.geometry("800x200")
  File "C:\Users\sunshine\anaconda3\lib\tkinter\__init__.py", line 1838, in wm_geometry
    return self.tk.call('wm', 'geometry', self._w, newGeometry)
_tkinter.TclError: can't invoke "wm" command: application has been destroyed
